<a href="https://colab.research.google.com/github/Nayeongseo/ML1-TeamProject-2/blob/main/%EC%97%90%ED%8F%AC%ED%81%AC%20-%20ica%20-%20%ED%91%9C%EC%A4%80%ED%99%94%20-%20svc%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import welch
import scipy.signal as signal
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import FastICA
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import CSP
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# train data의 input과 label 추출

In [ ]:
train_path = np.load('train.npy', allow_pickle='TRUE')

In [ ]:
print(f"Value: {train_path}")

Value: {'input': array([[[[  3.80859375,  -2.63671875, -10.64453125, ...,  -6.39648438,
           -2.34375   ,  -7.56835938],
         [  3.66210938,  -2.49023438,  -8.0078125 , ...,  -4.15039062,
           -2.39257812,  -7.66601562],
         [  2.9296875 ,  -5.22460938, -12.45117188, ...,  -7.2265625 ,
           -4.6875    , -10.88867188],
         ...,
         [  2.97851562,  -7.91015625,  -9.375     , ...,  -7.421875  ,
           -7.47070312, -15.57617188],
         [  4.73632812,  -6.93359375,  -8.93554688, ...,  -5.95703125,
           -5.61523438, -12.3046875 ],
         [  1.5625    ,  -9.22851562,  -9.08203125, ...,  -4.19921875,
           -5.76171875, -12.98828125]]],


       [[[ -6.20117188,   1.51367188,   4.00390625, ...,  -3.36914062,
           -8.83789062,  -9.91210938],
         [-10.44921875,  -5.61523438,   0.83007812, ...,  -9.1796875 ,
          -12.109375  , -14.11132812],
         [ -6.68945312,   0.53710938,   4.58984375, ...,  -5.859375  ,
          -10.

In [ ]:
# train_path에서 'input'과 'label' 데이터 추출
train_input = train_path.item()['input']
train_label = train_path.item()['label']

# 'input' 데이터의 형태 확인
print("Shape of 'input' data:", train_input.shape)

# 'input' 데이터를 4차원 배열로 변환
num_samples = train_input.shape[0]  # 샘플 수
num_channels = 1  # 채널 수 (EEG 데이터의 경우 채널이 하나이므로 1)
num_time_steps = train_input.shape[2]  # 시간 단계 수
num_features = train_input.shape[3]  # 특징 수

# 4차원 배열을 생성하여 데이터 복사
train_input_4d = np.zeros((num_samples, num_channels, num_time_steps, num_features))
for i in range(num_samples):
    train_input_4d[i, 0, :, :] = train_input[i, 0, :, :]

# 변환된 'input' 데이터의 형태 확인
print("Shape of 'input' data (4D):", train_input_4d.shape)

# 'label' 데이터의 형태 확인
print("Shape of 'label' data:", train_label.shape)

Shape of 'input' data: (4608, 1, 22, 1125)
Shape of 'input' data (4D): (4608, 1, 22, 1125)
Shape of 'label' data: (4608, 4)


In [ ]:
print(train_input_4d)

[[[[  3.80859375  -2.63671875 -10.64453125 ...  -6.39648438
     -2.34375     -7.56835938]
   [  3.66210938  -2.49023438  -8.0078125  ...  -4.15039062
     -2.39257812  -7.66601562]
   [  2.9296875   -5.22460938 -12.45117188 ...  -7.2265625
     -4.6875     -10.88867188]
   ...
   [  2.97851562  -7.91015625  -9.375      ...  -7.421875
     -7.47070312 -15.57617188]
   [  4.73632812  -6.93359375  -8.93554688 ...  -5.95703125
     -5.61523438 -12.3046875 ]
   [  1.5625      -9.22851562  -9.08203125 ...  -4.19921875
     -5.76171875 -12.98828125]]]


 [[[ -6.20117188   1.51367188   4.00390625 ...  -3.36914062
     -8.83789062  -9.91210938]
   [-10.44921875  -5.61523438   0.83007812 ...  -9.1796875
    -12.109375   -14.11132812]
   [ -6.68945312   0.53710938   4.58984375 ...  -5.859375
    -10.9375     -10.9375    ]
   ...
   [  9.1796875   12.45117188  17.72460938 ...   3.07617188
     -1.66015625  -4.98046875]
   [  9.13085938  13.33007812  16.74804688 ...   7.03125
      2.1484375   -0.

In [ ]:
train_input_3d = train_input_4d.squeeze()
print(train_input_3d.shape)

(4608, 22, 1125)


# test data의 input 추출

In [ ]:
test_path = np.load('test.npy', allow_pickle='TRUE')

In [ ]:
# 'input' 데이터만 추출하여 test 데이터 생성
test_4d = test_path.item().get('input')

# test 데이터의 형태 확인
print("Shape of 'test' data:", test_4d.shape)

Shape of 'test' data: (576, 1, 22, 1125)


In [ ]:
print(test_4d)

[[[[ -4.4921875   -5.859375     1.7578125  ...   9.03320312
      8.64257812   4.8828125 ]
   [ -3.515625    -5.37109375  -2.05078125 ...   7.56835938
      9.765625     5.51757812]
   [ -1.46484375  -5.078125     2.09960938 ...   7.2265625
      6.8359375    2.78320312]
   ...
   [  4.24804688   1.02539062   7.37304688 ...   3.80859375
      2.78320312   0.92773438]
   [  5.81054688   2.29492188   9.03320312 ...   4.24804688
      2.97851562   0.43945312]
   [  5.95703125   1.26953125  10.05859375 ...   4.296875
      3.85742188   1.31835938]]]


 [[[  0.43945312   5.41992188   5.02929688 ...  12.54882812
     11.86523438  13.42773438]
   [ -1.07421875   4.8828125    4.19921875 ...  13.0859375
      8.984375    12.20703125]
   [ -1.46484375   3.56445312   2.24609375 ...  12.3046875
     11.37695312  13.91601562]
   ...
   [-10.10742188  -1.46484375  -5.51757812 ...   3.75976562
      2.68554688   6.73828125]
   [ -9.765625    -3.36914062  -8.34960938 ...   3.07617188
      2.39257812 

In [ ]:
test = test_4d.squeeze()
print(test.shape)

(576, 22, 1125)


# 데이터 전처리

Bandpass Filter

In [ ]:
lowcut = 0.1
highcut = 30.0
fs = 250.0

def butter_bandpass(lowcut, hightcut, fs, order = 5):
  nyquist = 0.5*fs
  low = lowcut/nyquist
  high = highcut/nyquist
  b, a = signal.butter(order, [low, high], btype = 'band')
  return b, a

def bandpass_filter(data, lowcut, highcut, fs, order = 5):
  b, a = butter_bandpass(lowcut, highcut, fs, order = order)
  y = signal.filtfilt(b, a, data, axis = -1)
  return y

In [ ]:
train_input_filtered = np.zeros_like(train_input_3d)
for i in range(train_input_3d.shape[0]):
  for j in range(train_input_3d.shape[1]):
    train_input_filtered[i, j, :] = bandpass_filter(train_input_3d[i, j, :], lowcut, highcut, fs)

test_filtered = np.zeros_like(test)
for i in range(test.shape[0]):
  for j in range(test.shape[1]):
    test_filtered[i, j, :] = bandpass_filter(test[i, j, :], lowcut, highcut, fs)

In [ ]:
print(f'Train input filtered shape: {train_input_filtered.shape}')
print(f'Test filtered shape: {test_filtered.shape}')

Train input filtered shape: (4608, 22, 1125)
Test filtered shape: (576, 22, 1125)


ICA

에포크 추출

In [ ]:
# Verify the shapes before scaling
print("Train shape before scaling:", train_input_filtered.shape)
print("Test shape before scaling:", test_filtered.shape)

# Reshape for ICA
n_samples_train = train_input_filtered.shape[0]
n_samples_test = test_filtered.shape[0]
n_channels = train_input_filtered.shape[1]
n_time_steps = train_input_filtered.shape[2]

X_train_reshaped = train_input_filtered.reshape(n_samples_train, -1)  # (samples, features)
X_test_reshaped = test_filtered.reshape(n_samples_test, -1)  # (samples, features)

# Verify the reshaped shapes
print("Reshaped train shape:", X_train_reshaped.shape)
print("Reshaped test shape:", X_test_reshaped.shape)

# Apply ICA
ica = FastICA(n_components=22, random_state=0)
X_train_ica = ica.fit_transform(X_train_reshaped)

# Apply ICA transformation to test data
X_test_ica = ica.transform(X_test_reshaped)

# Reshape back to original dimensions
X_train_ica_reshaped = X_train_ica.reshape(n_samples_train, n_channels, -1)
X_test_ica_reshaped = X_test_ica.reshape(n_samples_test, n_channels, -1)

# Verify the final reshaped shapes
print("ICA transformed train shape:", X_train_ica_reshaped.shape)
print("ICA transformed test shape:", X_test_ica_reshaped.shape)

# Define epoch length
epoch_length = 100  # Number of time steps per epoch

# Function to segment data into epochs
def create_epochs(data, epoch_length):
    n_samples, n_channels, n_components = data.shape
    # Calculate number of epochs that can be extracted
    n_epochs = n_samples // epoch_length
    # Reshape data to (n_epochs, epoch_length, n_channels, n_components)
    epochs = data[:n_epochs * epoch_length].reshape(n_epochs, epoch_length, n_channels, n_components)
    return epochs

# Extract epochs from ICA-transformed data
train_epochs = create_epochs(X_train_ica_reshaped, epoch_length)
test_epochs = create_epochs(X_test_ica_reshaped, epoch_length)

# Verify the shapes of the epochs
print("Train epochs shape:", train_epochs.shape)
print("Test epochs shape:", test_epochs.shape)

# Verify the shapes before scaling
print("Train shape before scaling:", train_input_filtered.shape)
print("Test shape before scaling:", test_filtered.shape)

# Channel-wise scaling
scalers = {}
for i in range(train_input_filtered.shape[1]):  # iterate over channels
    scalers[i] = StandardScaler()
    train_input_filtered[:, i, :] = scalers[i].fit_transform(train_input_filtered[:, i, :].reshape(-1, 1)).reshape(train_input_filtered.shape[0], train_input_filtered.shape[2])

# Apply the same scalers to the test data
for i in range(test_filtered.shape[1]):
    test_filtered[:, i, :] = scalers[i].transform(test_filtered[:, i, :].reshape(-1, 1)).reshape(test_filtered.shape[0], test_filtered.shape[2])

# Verify the shapes after scaling
print("Train shape after scaling:", train_input_filtered.shape)
print("Test shape after scaling:", test_filtered.shape)

Train shape before scaling: (4608, 22, 1125)
Test shape before scaling: (576, 22, 1125)
Reshaped train shape: (4608, 24750)
Reshaped test shape: (576, 24750)


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA transformed train shape: (4608, 22, 1)
ICA transformed test shape: (576, 22, 1)
Train epochs shape: (46, 100, 22, 1)
Test epochs shape: (5, 100, 22, 1)
Train shape before scaling: (4608, 22, 1125)
Test shape before scaling: (576, 22, 1125)
Train shape after scaling: (4608, 22, 1125)
Test shape after scaling: (576, 22, 1125)


정규화 및 표준화

In [ ]:
# 표준화를 위한 객체 생성
standard_scaler = StandardScaler()

# 훈련 데이터 표준화
X_train_standardized = standard_scaler.fit_transform(X_train_reshaped)

# 테스트 데이터 표준화
X_test_standardized = standard_scaler.transform(X_test_reshaped)

# 결과 확인
print("Standardized train shape:", X_train_standardized.shape)
print("Standardized test shape:", X_test_standardized.shape)
print("Train epochs mean:", X_train_standardized.mean())
print("Train epochs std:", X_train_standardized.std())
print("Test epochs mean:", X_test_standardized.mean())
print("Test epochs std:", X_test_standardized.std())

Standardized train shape: (4608, 24750)
Standardized test shape: (576, 24750)
Train epochs mean: 2.7731900826102677e-18
Train epochs std: 1.0000000000000004
Test epochs mean: -0.0012732874283616282
Test epochs std: 0.951154106423943


특성 추출

In [ ]:
# 훈련 데이터의 차원 확인
print("Original train data shape:", X_train_standardized.shape)

# 훈련 데이터를 3D로 재구성
n_samples_train, n_features_train = X_train_standardized.shape
n_channels = 22  # 채널 수
n_time_steps = n_features_train // n_channels  # 시간 단계 수

X_train_3d = X_train_standardized.reshape(n_samples_train, n_channels, n_time_steps)

# 재구성된 데이터의 차원 확인
print("Reshaped train data shape:", X_train_3d.shape)


Original train data shape: (4608, 24750)
Reshaped train data shape: (4608, 22, 1125)


In [ ]:
def compute_csp_features_multiclass(train_input_filtered, test_filtered, train_labels, n_components=4):
    n_train_epochs, n_channels, n_times = train_input_filtered.shape
    n_test_epochs = test_filtered.shape[0]

    # CSP expects input shape (n_samples, n_channels, n_times)
    X_train = train_input_filtered.reshape(n_train_epochs, n_channels, n_times)
    X_test = test_filtered.reshape(n_test_epochs, n_channels, n_times)

    csp = CSP(n_components=n_components, reg=None, log=True)
    X_train_csp = []
    X_test_csp = []

    # Apply CSP for each class
    for i in range(train_labels.shape[1]):
        # Convert multi-class labels to binary
        binary_labels = (train_labels[:, i] == 1).astype(int)

        # Fit CSP on the training data for the current class
        csp.fit(X_train, binary_labels)

        # Apply CSP to the training and test data
        X_train_csp.append(csp.transform(X_train))
        X_test_csp.append(csp.transform(X_test))

    X_train_csp = np.concatenate(X_train_csp, axis=1)
    X_test_csp = np.concatenate(X_test_csp, axis=1)

    return X_train_csp, X_test_csp

# CSP 특성 추출
X_train_csp, X_test_csp = compute_csp_features_multiclass(X_train_3d, X_test_standardized, train_label)

# 다중 클래스 분류 모델 학습
clf = make_pipeline(StandardScaler(), SVC())
clf.fit(X_train_csp, np.argmax(train_label, axis=1))  # 다중 클래스 레이블로 변환

# 테스트 데이터 예측
test_predictions = clf.predict(X_test_csp)

# 결과 출력
print("테스트 데이터 예측:", test_predictions)

Computing rank from data with rank=None
    Using tolerance 47 (2.2e-16 eps * 22 dim * 9.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 47 (2.2e-16 eps * 22 dim * 9.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 47 (2.2e-16 eps * 22 dim * 9.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Es

In [ ]:
# ID 값을 생성합니다. 여기서는 예측 결과의 개수만큼의 ID를 0부터 순서대로 생성합니다.
ids = list(range(len(test_predictions)))

# pandas DataFrame을 생성합니다.
df = pd.DataFrame({'ID': ids, 'TARGET': test_predictions})

# DataFrame을 csv 파일로 저장합니다. 'predictions.csv'라는 파일명을 사용합니다.
df.to_csv('submissions.csv', index=False)